**Neural network**

*Generating data set for training*

In [ ]:
state=[]
action=[]
nextstate=[]

In [ ]:
state.append([0,0,0,0,0,0,0,0])
action.append(0)
nextstate.append([0,0,0,0,0,0,0,0])

**RUN THIS CODE STILL THE LENGTH OF STATE,ACTION,NEXT STATE BECOMES MORE THAN 20000**

In [ ]:
from os import wait
import random

def generate_input():
    cars_in_lane = [random.randint(0, 100) for _ in range(4)]  # 4 values from 0-30
    wait_times = [0,0,0,0]
    return cars_in_lane + wait_times

def rotate_list(lst, n):
    return lst[n:] + lst[:n]

def rotate_update(wait_times, nn_output):
    selected_lane = 0  # Always the first lane
    green_time = nn_output[0]
    carsinlane = wait_times[:4]
    carswaittime = wait_times[4:8]

    cars_per_second = 1
    cars_passed = min(carsinlane[selected_lane], cars_per_second * green_time)
    carsinlane[selected_lane] -= cars_passed
    carswaittime[selected_lane] = 0

    for i in range(4):
        if i != selected_lane:
            if carsinlane[i] > 0:
                carswaittime[i] += green_time
            else:
                carswaittime[i] = 0

    carsinlane = rotate_list(carsinlane, 1)
    carswaittime = rotate_list(carswaittime, 1)
    return carsinlane + carswaittime

def checkcross(greenlist,carsinlane):
  new_cars_wait_time=carsinlane[4:8]
  exceeded_times = [wait for wait in new_cars_wait_time if wait >= 60]
  diff_exceeded = [wait - 60 for wait in new_cars_wait_time if wait >= 60]
  if all(num == 0 for num in carsinlane[:4]):
    return False, 0
  else:
    if len(diff_exceeded)>0:
      return True,diff_exceeded
    else:
      return False,0


def correcttime(value, greenlist):
    i = 0
    value2=sum(greenlist)
    if value[0]<value2:
      while i < value[0]:
          for j in range(4):
              if greenlist[j] > 0:
                  greenlist[j] -= 1
                  i += 1
              if i == value[0]:  # Exit condition
                  break

    return greenlist




def calculate_green_times(carsinlane, total_green_time=120):
    total_cars = sum(carsinlane[:4])
    if total_cars==0:
      return [0,0,0,0]
    wait_times=carsinlane[4:8]
    if total_cars == 0:
        list1= [0] * 4
        return list1
    green_times = [(cars / total_cars) * total_green_time for cars in carsinlane[:4]]
    greenlist=[int(gt) for gt in green_times]
    for i in range(4):
      if greenlist[i]>carsinlane[i]:
        greenlist[i]=carsinlane[i]
    carsinlane2 = carsinlane
    for i in greenlist:
      updated_wait_times = rotate_update(carsinlane2, [i])
      carsinlane2=updated_wait_times
      status,value=checkcross(greenlist,carsinlane2)
      if status==True:
        listc=correcttime(value,greenlist)
      else:
        listc=greenlist
    return listc

# 🔹 Example usage
for _ in range(100):

  # 🔹 Example usage
  wait_times =  generate_input()  # Initial cars & wait times
  nn_output = calculate_green_times(wait_times)  # Lane 2 gets green light for 10 sec
  while any(wait_times[:4]):  # Continue until all car counts are 0
    for i in nn_output:
      updated_wait_times = rotate_update(wait_times, [i])
      state.append(wait_times)
      action.append(i)
      nextstate.append(updated_wait_times)
      wait_times=updated_wait_times
    if all(num == 0 for num in wait_times[:4]):
      break
    nn_output = calculate_green_times(wait_times)  # Lane 2 gets green light for 10 sec

print(len(state))
print(len(action))
print(len(nextstate))

**NN Structure and Training**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import random

# Define the neural network
class TrafficNN(nn.Module):
    def __init__(self):
        super(TrafficNN, self).__init__()
        self.fc1 = nn.Linear(8, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))  # Output in range (0,1)
        return x * 120  # Rescale to 0-120

# Initialize model, loss function, and optimizer
model = TrafficNN()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Better learning rate

# Convert lists to tensors (Ensure enough data)
X_train = torch.tensor(state[:20000], dtype=torch.float32) / 100  # Normalize inputs
y_train = torch.tensor(action[:20000], dtype=torch.float32).view(-1, 1) / 120  # Normalize outputs

# Training loop
epochs = 25000
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    if epoch % 500 == 0:  # Print loss every 500 epochs
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

# Test the model
test_state = torch.tensor([[10, 0, 0, 0, 5, 0, 0, 0]], dtype=torch.float32) / 100  # Normalize test input
test_state = test_state.view(1, -1)
predicted_action = model(test_state).item() * 120  # Rescale back to 0-120
print(f"Predicted Green Time: {predicted_action:.2f} seconds")


**TEST THE MODEL**

In [ ]:
X_test = torch.tensor(state, dtype=torch.float32) / 100  # Normalize inputs
y_test = torch.tensor(action, dtype=torch.float32).view(-1, 1) / 120  # Normalize outputs

In [ ]:
with torch.no_grad():
  y_eval=model.forward(X_test)
  loss=criterion(y_eval,y_test)
print(loss)

**SAVE THE MODEL**

In [ ]:
torch.save(model.state_dict(), "traffic_nn2.pth")

**LOAD SAVED MODEL**

In [ ]:
# Define the neural network
class TrafficNN(nn.Module):
    def __init__(self):
        super(TrafficNN, self).__init__()
        self.fc1 = nn.Linear(8, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))  # Output in range (0,1)
        return x * 120  # Rescale to 0-120
# Load the trained model
model2 = TrafficNN()
criterion = nn.MSELoss()
model2.load_state_dict(torch.load("traffic_nn2.pth"))
model2.eval()

# Generate initial state (cars in lanes and wait times)
ini=generate_input()
print("Begin:")
while any(ini[:4]):
  test_state = torch.tensor([ini], dtype=torch.float32) / 100  # Normalize test input
  print(ini)
  test_state = test_state.view(1, -1)  # Ensure correct shape
  predicted_action = round(model2(test_state).item() * 120 ) # Rescale prediction
  print(f"Predicted Green Time: {predicted_action:.2f} seconds")
  ini=rotate_update(ini, [predicted_action])
print(ini)
print("End:")